In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
import numpy as np
from keras.layers import Lambda
from keras import backend as K
from keras import regularizers

from keras.optimizers.legacy import SGD

In [2]:
import torch.nn as nn
from keras.callbacks import EarlyStopping

In [8]:
def AlexNet(num_classes=10):
    model = Sequential()

    # Feature extraction layers
    model.add(Conv2D(64, (3, 3), padding='same', input_shape=(32, 32, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(192, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(384, (3, 3), padding='same'))
    model.add(Activation('relu'))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dropout(0.6))
    model.add(Dense(2048))
    model.add(Activation('relu'))

    model.add(Dropout(0.6))
    model.add(Dense(2048))
    model.add(Activation('relu'))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    return model

In [9]:
alexnet = AlexNet(num_classes=10)

In [ ]:
from keras.optimizers import Adam

In [10]:
class cifar10vgg:
    def __init__(self, model, lr, train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32, 32, 3]

        self.model = model
        if train:
            self.model = self.train(self.model, lr)
        else:
            self.model.load_weights('cifar10vgg.h5')

    def normalize(self, X_train, X_test):
        mean = np.mean(X_train, axis=(0, 1, 2, 3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train - mean) / (std + 1e-7)
        X_test = (X_test - mean) / (std + 1e-7)
        return X_train, X_test

    def normalize_production(self, x):
        mean = 120.707
        std = 64.15
        return (x - mean) / (std + 1e-7)

    def predict(self, x, normalize=True, batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x, batch_size)

    def train(self, model, lr):
        batch_size = 128
        maxepoches = 250
        learning_rate = lr
        lr_decay = 1e-6
        lr_drop = 20

        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train, x_test = self.normalize(x_train, x_test)

        y_train = keras.utils.to_categorical(y_train, self.num_classes)
        y_test = keras.utils.to_categorical(y_test, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

        datagen = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True
        )
        datagen.fit(x_train)

        sgd = SGD(learning_rate=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

        historytemp = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                                steps_per_epoch=x_train.shape[0] // batch_size,
                                epochs=maxepoches,
                                validation_data=(x_test, y_test),
                                callbacks=[reduce_lr, early_stopping],
                                verbose=2)
        model.save_weights('cifar10vgg.h5')
        return model

In [11]:
if __name__ == '__main__':


    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    model = cifar10vgg(alexnet, 0.01)

    predicted_x = model.predict(x_test)
    residuals = np.argmax(predicted_x,1)!=np.argmax(y_test,1)

    loss = sum(residuals)/len(residuals)
    print("the validation 0/1 loss is: ",loss)

Epoch 1/250
390/390 - 30s - loss: 1.9107 - accuracy: 0.2762 - val_loss: 1.4887 - val_accuracy: 0.4507 - lr: 0.0100 - 30s/epoch - 76ms/step
Epoch 2/250
390/390 - 30s - loss: 1.4455 - accuracy: 0.4641 - val_loss: 1.3187 - val_accuracy: 0.5364 - lr: 0.0100 - 30s/epoch - 77ms/step
Epoch 3/250
390/390 - 28s - loss: 1.2195 - accuracy: 0.5576 - val_loss: 1.2557 - val_accuracy: 0.5833 - lr: 0.0100 - 28s/epoch - 72ms/step
Epoch 4/250
390/390 - 28s - loss: 1.0502 - accuracy: 0.6233 - val_loss: 0.8687 - val_accuracy: 0.6887 - lr: 0.0100 - 28s/epoch - 71ms/step
Epoch 5/250
390/390 - 28s - loss: 0.9302 - accuracy: 0.6705 - val_loss: 0.8699 - val_accuracy: 0.7028 - lr: 0.0100 - 28s/epoch - 71ms/step
Epoch 6/250
390/390 - 28s - loss: 0.8463 - accuracy: 0.7056 - val_loss: 0.8436 - val_accuracy: 0.7250 - lr: 0.0100 - 28s/epoch - 72ms/step
Epoch 7/250
390/390 - 28s - loss: 0.7813 - accuracy: 0.7295 - val_loss: 0.7005 - val_accuracy: 0.7653 - lr: 0.0100 - 28s/epoch - 72ms/step
Epoch 8/250
390/390 - 28s -

In [12]:
# Test accuracy calculation without using evaluate
correct_predictions = np.sum(np.argmax(predicted_x, axis=1) == np.argmax(y_test, axis=1))
test_accuracy = correct_predictions / len(y_test)
print("Test accuracy:", test_accuracy)

Test accuracy: 0.8724
